<p style="background-color:lightgreen;font-family:newtimeroman;font-size:23px;line-height:1.7em;text-align:center;border-radius:5px 5px">Sentiment_Analysis_[IMDB_Movie_Review]_Using_NLTK</p>

In [1]:
# import important modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# sklearn modules
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB # classifier 
from sklearn.metrics import (accuracy_score,classification_report,plot_confusion_matrix)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [3]:
# text preprocessing modules
from string import punctuation 

In [4]:
# text preprocessing modules
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re #regular expression

In [5]:
# Download dependency
for dependency in (
    "brown",
    "names",
    "wordnet",
    "averaged_perceptron_tagger",
    "universal_tagset",
):
    nltk.download(dependency)
    
import warnings
warnings.filterwarnings("ignore")
# seeding
np.random.seed(123)

[nltk_data] Error loading brown: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading names: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading averaged_perceptron_tagger: <urlo

In [6]:
df = pd.read_csv(r"C:\PYTHON\AI_ML\NLP\imdbdataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [7]:
df.shape

(50000, 2)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [10]:
# evalute news sentiment distribution
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

### Process the Data

In [11]:
stop_words =  stopwords.words('english')
def text_cleaning(text, remove_stop_words=True, lemmatize_words=True):
    # Clean the text, with the option to remove stop_words and to lemmatize word
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) # remove numbers
        
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if lemmatize_words:
        text = text.split()
        lemmatizer = WordNetLemmatizer() 
        lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(lemmatized_words)
    
    # Return a list of words
    return(text)

In [12]:
#clean the review
df["cleaned_review"] = df["review"].apply(text_cleaning)

In [13]:
#split features and target from  data 
X = df["cleaned_review"]
y = df.sentiment.values

In [14]:
# split data into train and validate
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.15,random_state=42,shuffle=True,stratify=y)

In [15]:
# Create a classifier in pipeline
sentiment_classifier = Pipeline(steps=[('pre_processing',TfidfVectorizer(lowercase=False)),('naive_bayes',MultinomialNB())])

In [16]:
# train the sentiment classifier 
sentiment_classifier.fit(X_train,y_train)

Pipeline(steps=[('pre_processing', TfidfVectorizer(lowercase=False)),
                ('naive_bayes', MultinomialNB())])

In [17]:
# test model performance on valid data 
y_preds = sentiment_classifier.predict(X_valid)
y_preds

array(['negative', 'negative', 'negative', ..., 'positive', 'positive',
       'positive'], dtype='<U8')

In [18]:
accuracy_score(y_valid,y_preds)

0.8708

In [19]:
v = classification_report(y_valid,y_preds)
print(v)

              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      3750
    positive       0.88      0.86      0.87      3750

    accuracy                           0.87      7500
   macro avg       0.87      0.87      0.87      7500
weighted avg       0.87      0.87      0.87      7500



### Save the Model Pipeline

In [20]:
#save model 
import joblib 
joblib.dump(sentiment_classifier, 'sentiment_model_pipeline.pkl')

['sentiment_model_pipeline.pkl']

### Real Time Prediction

In [21]:
# Reloading Model
test = joblib.load('sentiment_model_pipeline.pkl')

In [22]:
# Single row for testing our model 
final = X.head(1)

In [23]:
# Checking result
result = test.predict(final)

In [24]:
result

array(['positive'], dtype='<U8')

### Conclusion

** THANK YOU**